In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Dropout, Input

from sklearn.model_selection import train_test_split
from tokenizers import ByteLevelBPETokenizer

2024-11-09 17:15:16.468352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['Review'], vocab_size=1024, min_frequency=256)

In [4]:
samples['x'] = samples['Review'].apply(lambda x: tokenizer.encode(x).ids)
sequence_size = samples['x'].apply(len).max()
samples['x'] = samples['x'].apply(lambda x: (([0]*sequence_size)+x)[-sequence_size:])

sequence_size

5540

In [5]:
x = np.array(samples['x'].tolist())
y = np.array(samples['Sentiment'].tolist())

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((40000, 5540), (40000,), (10000, 5540), (10000,))

In [7]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256)
test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(256)

In [8]:
model = Sequential([
    Input(shape=(5540,)),
    Embedding(input_dim=1024, output_dim=32),
    Flatten(),
    Dropout(.25),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train, epochs=64, validation_data=test)

Epoch 1/64
157/157 [==============================] - 64s 404ms/step - loss: 0.6408 - accuracy: 0.6386 - val_loss: 0.4589 - val_accuracy: 0.7967
Epoch 2/64
157/157 [==============================] - 72s 460ms/step - loss: 0.3943 - accuracy: 0.8268 - val_loss: 0.3918 - val_accuracy: 0.8233
Epoch 3/64
157/157 [==============================] - 71s 453ms/step - loss: 0.3443 - accuracy: 0.8522 - val_loss: 0.3821 - val_accuracy: 0.8269
Epoch 4/64
157/157 [==============================] - 71s 452ms/step - loss: 0.3139 - accuracy: 0.8690 - val_loss: 0.3807 - val_accuracy: 0.8277
Epoch 5/64
157/157 [==============================] - 71s 454ms/step - loss: 0.2857 - accuracy: 0.8846 - val_loss: 0.3840 - val_accuracy: 0.8274
Epoch 6/64
157/157 [==============================] - 72s 461ms/step - loss: 0.2609 - accuracy: 0.8968 - val_loss: 0.3927 - val_accuracy: 0.8266
Epoch 7/64
157/157 [==============================] - 84s 532ms/step - loss: 0.2371 - accuracy: 0.9094 - val_loss: 0.4023 - val_ac